In [3]:
import torch

from project.models.TwoTower.SequenceEncoder import SequenceEncoder as SE

B, L, Emb = 2, 5, 32
genre_k = 3
model = SE(item_count = 100, genre_count = 10, emb_dim = Emb, max_seq_len = L)
model.eval()
history_seq = torch.LongTensor([
    [10, 20, 30, 3, 2],
    [0, 0, 3, 4, 1]
])
genre_sep = torch.randint(0, 10, (B, L, genre_k)) # 确保 padding 位置的 genre 也是 0 (虽然你的 embedding 会处理，但为了严谨)
genre_sep[history_seq == 0] = 0

print("输入形状构造完毕: History", history_seq.shape)


输入形状构造完毕: History torch.Size([2, 5])
